In [1]:
# Import necessary libraries for environment, agents, HTTP requests, and data modeling
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput, InputGuardrailTripwireTriggered
from typing import Dict, List, Optional
import os
from pydantic import BaseModel
import httpx

# Load environment variables from .env file
# override=True ensures existing variables are updated if present
load_dotenv(override=True)

In [ ]:
# Retrieve API keys and Discord webhook URL from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
discord_webhook_url = os.getenv('DISCORD_WEBHOOK_URL')

# Verify and print the status of each API key and webhook URL
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if discord_webhook_url:
    print(f"Discord Webhook URL exists and begins {discord_webhook_url[:8]}")
else:
    print("Discord Webhook URL not set")

In [ ]:
# Define instructions for social media content creation agents
instructions_professional = "You are a professional social media content creator working for 'Elite Digital', a marketing agency specializing in B2B tech clients. You craft precise, informative, and engaging social media posts for platforms like LinkedIn."

instructions_humorous = "You are a witty and engaging social media content creator working for 'Elite Digital'. You write humorous, trending, and shareable social media posts for platforms like X (formerly Twitter) and Instagram."

instructions_concise = "You are a busy social media content creator working for 'Elite Digital'. You write concise, to-the-point social media posts ideal for short-form content like Instagram Stories or quick updates."

In [4]:
# Define base URLs for OpenAI-compatible API endpoints
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [5]:
# Initialize AsyncOpenAI clients for different API providers
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

# Define models for each client
deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_8b_model = OpenAIChatCompletionsModel(model="llama3-8b-8192", openai_client=groq_client)

In [6]:
# Create agents for generating social media posts with different tones
post_agent_professional = Agent(name="Professional Post Agent", instructions=instructions_professional, model=deepseek_model)
post_agent_humorous = Agent(name="Humorous Post Agent", instructions=instructions_humorous, model=gemini_model)
post_agent_concise = Agent(name="Concise Post Agent", instructions=instructions_concise, model=llama3_8b_model)

In [7]:
# Define a common description for post generation tools
description_post_gen = "Generate a social media post for a given topic."

# Convert social media agents into tools
tool_professional_post = post_agent_professional.as_tool(tool_name="professional_post_generator", tool_description=description_post_gen)
tool_humorous_post = post_agent_humorous.as_tool(tool_name="humorous_post_generator", tool_description=description_post_gen)
tool_concise_post = post_agent_concise.as_tool(tool_name="concise_post_generator", tool_description=description_post_gen)

In [8]:
# Define Pydantic models for structured Discord webhook messages
class DiscordEmbedField(BaseModel):
    name: str
    value: str
    inline: Optional[bool] = False

class DiscordEmbed(BaseModel):
    title: Optional[str] = None
    description: Optional[str] = None
    color: Optional[int] = None
    fields: Optional[List[DiscordEmbedField]] = None

class DiscordWebhookMessage(BaseModel):
    content: Optional[str] = None
    username: Optional[str] = "Elite Digital Social Bot"
    avatar_url: Optional[str] = None
    embeds: Optional[List[DiscordEmbed]] = None
    tts: Optional[bool] = False

# Define a tool to send messages to Discord via webhook
@function_tool
async def send_discord_message(message_payload: DiscordWebhookMessage) -> Dict[str, str]:
    """Send a message to Discord via webhook with structured content."""
    if not discord_webhook_url:
        return {"status": "error", "message": "Discord webhook URL not set."}

    try:
        payload = message_payload.model_dump(exclude_none=True)
        print("Payload to Discord:", payload)  # Debug output

        if not payload.get("content") and not payload.get("embeds"):
            return {"status": "error", "message": "Empty message: no content or embeds provided."}

        response = await httpx.post(discord_webhook_url, json=payload)
        response.raise_for_status()

        return {"status": "success", "message": "Message sent to Discord successfully."}
    except httpx.RequestError as e:
        return {"status": "error", "message": f"Request error: {e}"}
    except httpx.HTTPStatusError as e:
        return {"status": "error", "message": f"HTTP error: {e.response.status_code} - {e.response.text}"}

In [9]:
# Define instructions for the Discord formatter agent
discord_formatter_instructions = """You are a Discord message formatter. You receive a social media post in text format and need to convert it into a structured JSON payload for a Discord webhook. The output must be an instance of the `DiscordWebhookMessage` Pydantic model. You should create a clear and compelling Discord message, potentially using embeds for better presentation. If a 'platform' is specified (e.g., 'LinkedIn', 'X', 'Instagram'), try to adapt the formatting for that platform within the Discord message (e.g., add relevant emojis, hashtags). Focus on making the message visually appealing and easy to read on Discord."""

# Create the Discord formatter agent
discord_formatter = Agent(
    name="Discord Message Formatter",
    instructions=discord_formatter_instructions,
    output_type=DiscordWebhookMessage,
    model="gpt-4o-mini"
)

# Convert the formatter agent into a tool
discord_format_tool = discord_formatter.as_tool(
    tool_name="format_for_discord",
    tool_description="Convert a raw social media post text into a structured Discord webhook message payload."
)

In [10]:
# Define tools for the Discord dispatcher
discord_tools = [discord_format_tool, send_discord_message]

# Define instructions for the Discord dispatcher agent
discord_dispatcher_instructions = """You are a Discord message dispatcher. You receive a final social media post. Your task is to first use the 'format_for_discord' tool to convert the post into a structured Discord message payload, and then use the 'send_discord_message' tool to send it to the designated Discord channel."""

# Create the Discord dispatcher agent
discord_dispatcher_agent = Agent(
    name="Discord Dispatcher",
    instructions=discord_dispatcher_instructions,
    tools=discord_tools,
    model="gpt-4o-mini",
    handoff_description="Format a social media post for Discord and send it."
)

In [11]:
# Define tools and handoff agents for the Social Media Manager
post_generation_tools = [tool_professional_post, tool_humorous_post, tool_concise_post]
handoff_agents = [discord_dispatcher_agent]

In [13]:
# Define instructions for the Social Media Manager agent
social_media_manager_instructions = """
You are a Social Media Manager for 'Elite Digital'. Your task is to create compelling social media posts using 3 specialized agents:
- A professional tone agent
- A humorous tone agent
- A concise tone agent

You must:
1. Use each of the 3 social media tools exactly once.
2. Compare the 3 outputs.
3. Select the single best one using your judgment of which is most effective and suitable for the given platform.
4. Handoff the selected post to the 'Discord Dispatcher' agent for formatting and sending.

Do not loop or retry. Only call each tool once.
"""

# Create the Social Media Manager agent
social_media_manager = Agent(
    name="Social Media Manager",
    instructions=social_media_manager_instructions.strip(),
    tools=post_generation_tools,
    handoffs=handoff_agents,
    model="gpt-4o-mini"
)

# Define a sample message topic for generating a social media post
message_topic = "Generate a social media post about the benefits of AI in cybersecurity for LinkedIn."

# Run the Social Media Manager to generate and process the post
with trace("Automated Social Media Post") as t:
    result = await Runner.run(social_media_manager, message_topic)
print(result.final_output)

In [ ]:
# Define a Pydantic model for keyword check output
class KeywordCheckOutput(BaseModel):
    contains_sensitive_keyword: bool
    found_keywords: List[str]

# Create an agent to check for sensitive keywords
guardrail_keyword_agent = Agent(
    name="Keyword Check Guardrail",
    instructions="Check if the user's request contains any sensitive or restricted keywords related to controversial topics, competitive names, or client confidential information. Respond with a boolean indicating if such keywords were found and a list of the keywords if any.",
    output_type=KeywordCheckOutput,
    model="gpt-4o-mini"
)

# Define an input guardrail to block sensitive keywords
@input_guardrail
async def guardrail_against_sensitive_keywords(ctx, agent, message):
    """
    A guardrail to check if the user's input message contains any sensitive keywords.
    Trips the wire if sensitive keywords are detected.
    """
    result = await Runner.run(guardrail_keyword_agent, message, context=ctx.context)
    contains_sensitive = result.final_output.contains_sensitive_keyword
    return GuardrailFunctionOutput(
        output_info={"found_keywords": result.final_output.found_keywords},
        tripwire_triggered=contains_sensitive
    )

In [16]:
# Create a Social Media Manager agent with input guardrail
careful_social_media_manager = Agent(
    name="Careful Social Media Manager",
    instructions=social_media_manager_instructions,
    tools=post_generation_tools,
    handoffs=handoff_agents,
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_sensitive_keywords]
)

# Test message expected to trip the guardrail
message_guarded_fail = "Create a social media post about why 'CompetitorX' is inferior for B2B cybersecurity."

print("\n--- Running with Guardrail (should be tripped) ---")
with trace("Protected Automated Social Media Post - Fail"):
    try:
        result_guarded_fail = await Runner.run(careful_social_media_manager, message_guarded_fail)
        print(f"Guarded Result (Fail): {result_guarded_fail}")
    except InputGuardrailTripwireTriggered as e:
        print(f"❌ Guardrail Tripped! Input blocked for: '{message_guarded_fail}'")
        print(f"🔒 Guardrail Name: {e.guardrail_result.guardrail.get_name()}")
        print("📋 Full Guardrail Result:", e.guardrail_result)

# Test message expected to pass the guardrail
message_guarded_success = "Generate a social media post about the importance of data privacy in cloud computing."

print("\n--- Running with Guardrail (should succeed) ---")
with trace("Protected Automated Social Media Post - Success"):
    try:
        result_guarded_success = await Runner.run(careful_social_media_manager, message_guarded_success)
        print(f"✅ Guarded Result (Success): {result_guarded_success}")
    except InputGuardrailTripwireTriggered as e:
        print(f"❌ Unexpected Guardrail Trip for: '{message_guarded_success}'")
        print(f"🔒 Guardrail Name: {e.guardrail_result.guardrail.get_name()}")
        print("📋 Full Guardrail Result:", e.guardrail_result)